### Instalação dos modulos para o uso do projeto

In [ ]:
!pip install pytest
!pip install unittest
!pip install tika #necessita da instalação de java
!pip install PyPDF2
!pip install python-docx
!pip install -U spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_md
!python -m spacy download pt_core_news_lg

### Importação dos modulos 

In [ ]:
import os
import re
import warnings
import PyPDF2
import pandas as pd
import string
import nltk
import pytest
import unittest
import spacy #NLP e Machine learning
from PyPDF2 import PdfFileWriter, PdfFileReader
from tqdm import tqdm_notebook
from sklearn import utils
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize,sent_tokenize
from string import punctuation
from tika import parser
from docx import Document
import unicodedata
warnings.filterwarnings("ignore")

### Entrada dos arquivos para a extração dos textos

In [ ]:
def import_arquivos(diretorio):
    arquivos = [arq for arq in os.listdir(diretorio) if (arq.lower().endswith(".pdf") | arq.lower().endswith(".docx"))]
    return(arquivos)

In [ ]:
### Inserção dos textos em objeto python

In [ ]:
def object_create(diretorio):
    arquivos = import_arquivos(diretorio)
    textos = []
    nomes = []
    erros = []
    for arq in arquivos:
        docx = []
        if (arq.endswith(".docx")):
            aux = Document(arq)
            nomes.append(arq) 
            for doc in aux.paragraphs:
                temp = doc.text
                temp =re.sub('\s+', ' ', unicodedata.normalize("NFKD",temp)
                             .encode('ASCII','ignore')
                             .decode('ASCII')).lower().strip()
                docx.append(temp)
            textos.append(' '.join(docx))              
        else:
            try:
                temp= parser.from_file(arq)['content']
                temp = temp.lower()
                temp = temp.replace("\n"," ")
                nomes.append(arq)
                textos.append(temp)
            except Exception as e:
                try:
                    temp = open(arq, 'rb')#insere no python
                    pdf = PyPDF2.PdfFileReader(temp)#cria objeto 
                    for j in range (pdf.getNumPages()):
                        temp = pdf.getPage(j).extractText()#retira o texto
                        temp = temp.lower()
                        temp = temp.replace("\n"," ")
                        temp = temp.replace("   "," ")
                        nomes.append(arq)
                        textos.append(temp)
                except Exception as e:
                      erros.append([e,arq])
                continue      
    return textos,nomes,erros   

In [ ]:
def limpeza(diretorio):
    textos, titulos, erros = object_create(diretorio)
    stop_words = stopwords.words('portuguese') + list(punctuation)
    for t in range(len(textos)):
        textos[t] = ' '.join([word for word in textos[t].split(" ") if word not in stop_words])
    return textos, titulos #,erros

In [ ]:
def similaridade(diretorio):
    textos, titulos = limpeza(diretorio)
    similaridade = pd.DataFrame(columns = ['Título_1', "Similaridade",'Título_2'])
    Título_1 = []
    Título_2 = []
    Similaridade = []
    nlp = spacy.load ("pt_core_news_md")
    for i in range(len(textos)):
        for j in range(len(textos)):
            Título_1.append(titulos[i])
            Título_2.append(titulos[j])
            Similaridade.append(nlp(textos[i]).similarity(nlp(textos[j])))
    similaridade['Título_1'] = Título_1
    similaridade['Título_2'] = Título_2
    similaridade["Similaridade"] = Similaridade
    similaridade.to_excel(diretorio +'/textos_similaridade.xlsx')

In [ ]:
similaridade('C:/Users/Vanessa/Downloads/INF2102')